<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-settings" data-toc-modified-id="Imports-and-settings-1">Imports and settings</a></span></li><li><span><a href="#Encapsulated-func-(obsolete)" data-toc-modified-id="Encapsulated-func-(obsolete)-2">Encapsulated func (obsolete)</a></span></li><li><span><a href="#Core" data-toc-modified-id="Core-3">Core</a></span><ul class="toc-item"><li><span><a href="#Interactive-template" data-toc-modified-id="Interactive-template-3.1">Interactive template</a></span></li><li><span><a href="#Dispersion-interactive" data-toc-modified-id="Dispersion-interactive-3.2">Dispersion interactive</a></span></li><li><span><a href="#Helper-function" data-toc-modified-id="Helper-function-3.3">Helper function</a></span></li><li><span><a href="#Mine-is-working!" data-toc-modified-id="Mine-is-working!-3.4">Mine is working!</a></span></li></ul></li></ul></div>

# Imports and settings

In [2]:
from NearFieldOptics.Materials import *
from NearFieldOptics.Materials.material_types import *
from TransferMatrixMethod import MatrixBuilder,Calculator
import sympy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import scipy.special as ss
import mpmath as mp
from mpmath.ctx_fp import FPContext
from sys import stdout
sympy.init_printing(use_unicode=True)

<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Bi2Se3_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "PMMA_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "sio2_300nm_extracted_epsilon_cone_A=2a.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "TaS2_eps_230K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "TaS2_eps_30K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Erik_BSTS_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "VO2_Insulating.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "VO2_Metallic.pickle"...


In [3]:
from bokeh.io import *
from bokeh.layouts import *
from bokeh.plotting import *
from bokeh.models.renderers import *
from bokeh.palettes import *
from bokeh.models.widgets import *
from bokeh.models import *
from bokeh import events

from NearFieldOptics.Materials import *
from NearFieldOptics import TipModels as T
import copy
import numpy as np
from matplotlib import pyplot as plt

output_notebook()
# output_file("new GUI layout.html")

Loading BokehJS ...

# Encapsulated func (obsolete)

In [3]:
def calculate_and_plot(freqs,sample,quantity,graph):
    if quantity.value=="near-field signal":
        ##no normalization
        calculated=T.LightningRodModel(freqs,rp=sample.reflection_p,a=a,Nqs=Nqs,Nzs=Nzs,\
                                            amplitude=amplitude,normalize_to=None,\
                                            normalize_at=normFrequency)
        global nfs
        nfs=graph.line(freqs,abs(calculated['signal_3']),legend="s3",line_color="#0093dd",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    elif quantity.value=="far-field reflectivity (p-polarized)":
        ##picked angle 0 as default, generalize
        rp_farfield=sample.reflection_p(freqs,angle=0)
        global fReflecP
        fReflecP=graph.line(freqs,abs(rp_farfield),legend="far field (p) amplitude",line_color="#0093dd",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    elif quantity.value=="far-field reflectivity (s-polarized)":
        ##picked angle 0 as default, generalize
        rs_farfield=sample.reflection_s(freqs,angle=0)
        global fReflecS
        fReflecS=graph.line(freqs,abs(rs_farfield),legend="far field (s) amplitude",line_color="#0093dd",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    #need to add far-field transmission

    elif quantity.value=="permittivity":
        eps=sample.epsilon(freqs)
        global rl
        rl=graph.line(freqs,eps.real,legend="real",line_color="#0093dd",line_width=2)
        global img
        img=graph.line(freqs,eps.imag,legend="imaginary",line_color="orange",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    elif quantity.value=="near-field reflectivity":
        rp_nearfield=sample.reflection_p(freqs,angle=None,q=1/a)
        global nReflec
        nReflec=graph.line(freqs,abs(rp_nearfield),legend="near field amplitude",line_color="orange",line_width=2)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

    else:
        #plot a thick red cross on panel
        x1 = np.linspace(0, 10, 100)
        y1 = x1
        y2 = 10-x1
        global x
        x=graph.line(x1, y1, legend="ERROR", line_color="red", line_width=6)
        global xx
        xx=graph.line(x1, y2, legend="ERROR", line_color="red", line_width=6)
        graph.title.text=exitMaterial.value
        graph.title.align="center"

# Core

## Interactive template

In [4]:
# def modify_doc(doc):
    
#     material_list = ["SiO2_300nm","SingleLayerGraphene","another"]
    
#     materialMenu = Select(title="Material", options=sorted(material_list), value="SiO2_300nm",width=200)
    
#     thicknessText = TextInput(title="Thickness (nm):", value="0")
#     muText = TextInput(title="Chemical potential (cm-1):", value="1000")
    
#     select_code = """
#     to_show = cb_obj.value;
#     options = cb_obj.options

#     for (i=0;i<options.length;i++) {
#         box = document.getElementsByClassName(options[i])[0];
#         if (!box.className.includes("hidden")) {box.className+=" hidden";}
#         if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
#     }
#     """
    
#     materialMenu.callback = CustomJS(code=select_code)
    
#     box01 = widgetbox(materialMenu)
#     box1 = widgetbox(muText,css_classes = ['SingleLayerGraphene','another','hidden'])
#     box2 = widgetbox(thicknessText,css_classes = ['SiO2_300nm','another'])
    
#     l = layout(box01,box1,box2)
#     doc.add_root(l)
    
# show(modify_doc)

## Dispersion interactive

In [5]:
# def modify_doc(doc):
#     dispersion_button_group = Select(title="Part to calculate:", options=[
#         "dispersion","frequency_line_cut", "q_line_cut"],value="dispersion")

#     dispersionMinFreq = TextInput(title="Min frequency: ",value="200")
#     dispersionMaxFreq = TextInput(title="Max frequency: ",value="2000")
#     dispersionFreqNum = TextInput(title="# of frequency points: ",value="300")
#     dispersionMinQ = TextInput(title="Min in-plane momentum (q): ",value="1")
#     dispersionMaxQ = TextInput(title="Max q: ",value="1e6")
#     dispersionQNum = TextInput(title="# of q: ",value="300")
    
#     minFreq = TextInput(title="Min frequency: ",value="200")
#     maxFreq = TextInput(title="Max frequency: ",value="2000")
#     freqNum = TextInput(title="# of frequency points: ",value="300")
#     qValue = TextInput(title="q value: ",value="1/30e-7")
    
#     minQ = TextInput(title="Min in-plane momentum (q): ",value="1")
#     maxQ = TextInput(title="Max q: ",value="1e6")
#     qNum = TextInput(title="# of q: ",value="300")
#     freqValue = TextInput(title="Frequency value: ",value="1000")
    
#     dispersion_code = """
#     to_show = cb_obj.value;
#     options = cb_obj.options

#     for (i=0;i<options.length;i++) {
#         box = document.getElementsByClassName(options[i])[0];
#         if (!box.className.includes("hidden")) {box.className+=" hidden";}
#         if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
#     }
#     """
#     dispersion_button_group.callback = CustomJS(code=dispersion_code)
    
#     dBox = column(row(dispersionMinFreq, dispersionMaxFreq,dispersionFreqNum),
#                   row(dispersionMinQ, dispersionMaxQ, dispersionQNum), css_classes = ['dispersion'])
#     fBox = column(row(minFreq,maxFreq,freqNum),qValue, css_classes = ['frequency_line_cut','hidden'])
#     qBox = column(row(minQ, maxQ, qNum),freqValue, css_classes = ['q_line_cut','hidden'])
    
# #     dfBox = column(dBox,fBox,css_classes = ['dispersion(and)freq','hidden'])
    
#     l = layout(dispersion_button_group,dBox,fBox,qBox)
#     doc.add_root(l)
    
# show(modify_doc)

## Helper function

In [4]:
def _get_name_of_LM(layerList,computerLayerList):
    string = ''
    for i in range(0,len(layerList)-1,1):
        if type(computerLayerList[i])==SingleLayerGraphene:
            string = string+layerList[i][0]+' - '
        else:
            string = string+layerList[i][0] + '(' + str(int(layerList[i][1])) + 'nm)' + ' - '

    if type(computerLayerList[len(layerList)-1])==SingleLayerGraphene:
        string = string+layerList[len(layerList)-1][0]
    else:
        string = string+layerList[len(layerList)-1][0] + '(' + str(int(layerList[len(layerList)-1][1])) + 'nm)'
    return string

In [5]:
def _get_computer_material_(matString):
        
        if matString == "SiO2":
            return SiO2_300nm
        elif matString == "BN_Caldwell":
            return BN_Caldwell
        elif matString == "FePO4":
            return FePO4
        elif matString == "TaS2_CCDW":
            return TaS2_CCDW
        elif matString == "TaS2_NCCDW":
            return TaS2_NCCDW
        elif matString == "Silicon":
            return Si
        elif matString == "Air":
            return Air
        elif matString == "Au":
            return Au
        else:
            Logger.raiseException('Material not recognized.', exception=valueError)
            return None

In [6]:
def _get_index_list_(layerList):
    numList = list(range(1,len(layerList)+1,1))
    indexList = list()
    for i in numList:
        indexList.append(str(i))
    return indexList

## Mine is working! 
NOTE: js doesn't recognize string with space! have to use _

NOTE: the lack of comma in layout boxes towards the end is intentional

layerList

LM = LayeredMediaTM(layerArrayGUIInput = computerLayerList, entrance=entranceMaterial,exit=exitMaterial)

from Material input tab

In [7]:
layerList = [('BN_Caldwell', 30.0),('SingleLayerGraphene', '1000', '0.1'),('BN_Caldwell', 30.0)]
computerLayerList = [(BN_Caldwell,30e-7),SingleLayerGraphene(chemical_potential=1000,gamma=0.1),(BN_Caldwell,30e-7)]
string = _get_name_of_LM(layerList,computerLayerList)
# len(layerList)

In [8]:
string

'BN_Caldwell(30nm) - SingleLayerGraphene - BN_Caldwell(30nm)'

In [9]:
entranceMaterial = Air
exitMaterial = Si
LM = LayeredMediaTM(layerArrayGUIInput = computerLayerList)

In [18]:
def modify_doc(doc):
    
    # graphs
    s1 = figure(plot_width=500, plot_height=300)
    s1.xgrid.grid_line_color = None
    s1.ygrid.grid_line_color = None

    s2 = figure(plot_width=500, plot_height=300)
    s2.xgrid.grid_line_color = None
    s2.ygrid.grid_line_color = None

    s3 = figure(plot_width=500, plot_height=300)
    s3.xgrid.grid_line_color = None
    s3.ygrid.grid_line_color = None

    s4 = figure(plot_width=500, plot_height=300)
    s4.xgrid.grid_line_color = None
    s4.ygrid.grid_line_color = None
    
    # standard widgets
    if len(layerList)==1 and type(computerLayerList[0])!=type(SingleLayerGraphene()):
        quantity_list = ["permittivity",
                         "reflection_coefficient(dispersion)","reflection_coefficient(single_f)","reflection_coefficient(single_q)",
                         "transmission_coefficient(dispersion)","transmission_coefficient(single_f)","transmission_coefficient(single_q)",
                         "H_field(dispersion)","H_field(single_f)","H_field(single_q)",
                         "Coulomb_kernel(dispersion)","Coulomb_kernel(single_f)","Coulomb_kernel(single_q)",
                         "s0_signal","s1_signal","s2_signal","s3_signal","s4_signal"]
    else:
        quantity_list = ["reflection_coefficient(dispersion)","reflection_coefficient(single_f)","reflection_coefficient(single_q)",
                         "transmission_coefficient(dispersion)","transmission_coefficient(single_f)","transmission_coefficient(single_q)",
                         "H_field(dispersion)","H_field(single_f)","H_field(single_q)",
                         "Coulomb_kernel(dispersion)","Coulomb_kernel(single_f)","Coulomb_kernel(single_q)",
                         "s0_signal","s1_signal","s2_signal","s3_signal","s4_signal"]
    quantityMenu = Select(title="Quantity to calculate:", options=quantity_list, value="reflection_coefficient(dispersion)")
    partMenu = Select(title="Part to calculate:", options=["real","imaginary","absolute","angle"], value="absolute")
    polarizationMenu = Select(title="Polarization", options=['p','s'], value='p')
    plotPanel = RadioButtonGroup(
        labels=["Plot in 1st panel", "Plot in 2nd panel", "Plot in 3rd panel","Plot in 4th panel"], active=0)
    run = Button(label="Calculate and plot!", button_type="success")
    clear = Button(label="Clear graph",button_type="warning")
    
    #dispersion-related input widgets
    minFreq = TextInput(title="Min frequency: ",value="200")
    maxFreq = TextInput(title="Max frequency: ",value="2000")
    freqNum = TextInput(title="# of frequency points: ",value="300")
    singleQ = TextInput(title="q value: ",value="333333")
    
    minQ = TextInput(title="Min in-plane momentum (q): ",value="1")
    maxQ = TextInput(title="Max q: ",value="1e6")
    qNum = TextInput(title="# of q: ",value="300")
    singleFreq = TextInput(title="Frequency value: ",value="1000")
    
    minQuantityValue = TextInput(title="Min quantity value (currently only for dispersion): ",value="0")
    maxQuantityValue = TextInput(title="Max quantity value (currently only for dispersion): ",value="1e-5")
    
    #H field and kernel input widget
    indexList = _get_index_list_(layerList)
    index = Select(title="Index of interface (1 denotes the interface between entrance and first layer): ",
                   value="1",options=indexList)
    
    #s-SNOM signal input widgets
    size = TextInput(title="Tip apex radius size (nm): ",value="10")
    oscAmplitude = TextInput(title="Tip oscillation amplitude (nm): ",value="80")
    SNOMqNum = TextInput(title="# of q points: ",value="244")
    zNum = TextInput(title="# of z points: ",value="40")
    normFreq = TextInput(title="Normalization frequency (cm^-1): ",value="200")
    material_list = ["Au","SiO2_300nm","BN_Caldwell","Si_80K","Si_150K","Si_300K","FePO4","TaS2_CCDW","TaS2_NCCDW","Silicon","Air"]
    normMat = Select(title="Normalization material:", value="Au", options=material_list)
    
    quantity_code = """
    to_show = cb_obj.value;
    options = cb_obj.options

    for (i=0;i<options.length;i++) {
        box = document.getElementsByClassName(options[i])[0];
        if (!box.className.includes("hidden")) {box.className+=" hidden";}
        if (box.className.includes(to_show)) {box.className=box.className.replace(" hidden","");}
    }
    """
    
    quantityMenu.callback = CustomJS(code=quantity_code)
    
    #Add a layer
    def calculate_parameters():
        global graph
        
        try:
            if quantityMenu.value[3:]=="signal":

                minFreqValue = int(minFreq.value)
                maxFreqValue = int(maxFreq.value)
                freqNumValue = int(freqNum.value)
                freqs = np.linspace(minFreqValue,maxFreqValue,freqNumValue)

                a=float(size.value)*1e-7
                amplitude=float(oscAmplitude.value)*1e-7
                Nqs=int(SNOMqNum.value)
                Nzs=int(zNum.value)
                normFrequency=float(normFreq.value)
                matString=normMat.value
                normMaterial = _get_computer_material_(matString)

                calculated=T.LightningRodModel(freqs,rp=LM.reflection_p,a=a,Nqs=Nqs,Nzs=Nzs,\
                                                    amplitude=amplitude,normalize_to=normMaterial.reflection_p,\
                                                    normalize_at=normFrequency)
                
                #plotting
                graph = figure(x_axis_label='frequency (cm-1)',plot_width=500, plot_height=300)
                graph.title.text=exitMaterial.value
                graph.title.align="center"
                if quantityMenu.value=="s0_signal":
                    graph.line(freqs,abs(calculated['signal_0']),legend="s0",line_color="#0093dd",line_width=2)
                elif quantityMenu.value=="s1_signal":
                    graph.line(freqs,abs(calculated['signal_1']),legend="s1",line_color="#0093dd",line_width=2)
                elif quantityMenu.value=="s2_signal":
                    graph.line(freqs,abs(calculated['signal_2']),legend="s2",line_color="#0093dd",line_width=2)
                elif quantityMenu.value=="s3_signal":
                    graph.line(freqs,abs(calculated['signal_3']),legend="s3",line_color="#0093dd",line_width=2)
                elif quantityMenu.value=="s4_signal":
                    graph.line(freqs,abs(calculated['signal_4']),legend="s4",line_color="#0093dd",line_width=2)
                else:
                    Logger.raiseException('Invalid quantity menu value.',exception=ValueError)
                
            elif quantityMenu.value=="permittivity":
                minFreqValue = float(minFreq.value)
                maxFreqValue = float(maxFreq.value)
                freqNumValue = int(freqNum.value)
                freq = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
                eps=LM.epsilon(freq)
                
                graph = figure(x_axis_label='frequency (cm-1)',plot_width=500, plot_height=300)
                graph.line(freq,eps.real,legend="real",line_color="#0093dd",line_width=2)
                graph.line(freq,eps.imag,legend="imaginary",line_color="orange",line_width=2)
                graph.title.text=exitMaterial.value
                graph.title.align="center"

            else:
                polarization = polarizationMenu.value
                T = MatrixBuilder.TransferMatrix(LM,polarization=polarization)
                C = Calculator.Calculator(T)

                # obtain appropriate dimension of freq and q
                if quantityMenu.value[-12:]=="(dispersion)":
                    minFreqValue = float(minFreq.value)
                    maxFreqValue = float(maxFreq.value)
                    freqNumValue = int(freqNum.value)
                    freqIn = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
                    minQValue = float(minQ.value)
                    maxQValue = float(maxQ.value)
                    qNumValue = int(qNum.value)
                    qIn = np.linspace(minQValue,maxQValue,qNumValue) 
                    q,freq = np.meshgrid(qIn,freqIn)
                elif quantityMenu.value[-10:]=="(single_f)":
                    freq = float(singleFreq.value)
                    minQValue = float(minQ.value)
                    maxQValue = float(maxQ.value)
                    qNumValue = int(qNum.value)
                    q = np.linspace(minQValue,maxQValue,qNumValue)    
                elif quantityMenu.value[-10:]=="(single_q)":
                    q = float(singleQ.value)
                    minFreqValue = float(minFreq.value)
                    maxFreqValue = float(maxFreq.value)
                    freqNumValue = int(freqNum.value)
                    freq = np.linspace(minFreqValue,maxFreqValue,freqNumValue)
                else:
                    Logger.raiseException('Invalid quantity menu value.',exception=ValueError)

                # calculate the optical quantities (based on the previous freq and q)
                if quantityMenu.value[:10]=="reflection":
                    graphTitle = "r_"+polarizationMenu.value
                    C.assemble_analytical_reflection_coefficient()
                    result = C.get_numerical_reflection_coefficient(freq,q)
                elif quantityMenu.value[:12]=="transmission":
                    graphTitle = "t_"+polarizationMenu.value
                    C.assemble_analytical_transmission_coefficient()
                    result = C.get_numerical_transmission_coefficient(freq,q)
                elif quantityMenu.value[:7]=="H_field":
                    graphTitle = "H field"
                    n = int(index.value)
                    C.assemble_analytical_H_field(n, "before")
                    result = C.get_numerical_H_field(freq, q)       
                elif quantityMenu.value[:7]=="Coulomb":
                    graphTitle = "Coulomb kernel"
                    n = int(index.value)
                    C.assemble_analytical_kernel(n, "before")
                    result = C.get_numerical_kernel(freq, q)
                else:
                    Logger.raiseException('Invalid quantity menu value.',exception=ValueError)

                # get the part (real/imag/abs/angle) of the calculated quantities
                if partMenu.value=="real":
                    result = np.real(result)
                elif partMenu.value=="imaginary":
                    result = np.imag(result)
                elif partMenu.value=="absolute":
                    result = abs(result)
                elif partMenu.value=="angle":
                    result = np.angle(result)
                else:
                    Logger.raiseException('Invalid part menu value.',exception=ValueError)

                #create graph
                graph = figure(plot_width=500, plot_height=300)
                if type(freq)==float:
                    xRange = q
                    graph.line(xRange,result,legend=partMenu.value,line_color="#0093dd",line_width=2)
                    graph.yaxis.axis_label = graphTitle
                    graph.xaxis.axis_label = 'q (cm$^{-1}$)'
                    graph.title.text=_get_name_of_LM(layerList,computerLayerList)
                    graph.title.align="center"
                elif type(q)==float:
                    xRange = freq
                    graph.line(xRange,result,legend=partMenu.value,line_color="#0093dd",line_width=2)
                    graph.yaxis.axis_label = graphTitle
                    graph.xaxis.axis_label = 'Frequency (cm$^{-1}$)'
                    graph.title.text=_get_name_of_LM(layerList,computerLayerList)
                    graph.title.align="center"
                elif type(freq)==np.ndarray and type(q)==np.ndarray:
                    graph = figure(toolbar_location=None)
#                     graph = figure(x_range=(qIn.min(),qIn.max()), \
#                                    y_range=(freqIn.min(),freqIn.max()), toolbar_location=None)
                    graph.y_range = Range1d(freqIn.min(),freqIn.max())
                    graph.x_range = Range1d(qIn.min(),qIn.max())
                    graph.add_layout(Title(text=graphTitle+' dispersion', text_font_style="italic"), 'above')
                    graph.title.align="center"
                    graph.add_layout(Title(text=_get_name_of_LM(layerList,computerLayerList)), 'above')
                    graph.title.align="center"
                    
                    color_mapper = LinearColorMapper(palette="Viridis256", \
                                                     low=float(minQuantityValue.value), high=float(maxQuantityValue.value))
                    graph.image(image=[result], color_mapper=color_mapper,
                               dh=[freqIn.max()], dw=[qIn.max()], x=[qIn.min()], y=[freqIn.min()])
                    graph.xaxis.axis_label = 'q (cm-1)'
                    graph.yaxis.axis_label = 'freq (cm-1)'
                    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                                         label_standoff=12, border_line_color=None, location=(0,0))
                    graph.add_layout(color_bar, 'right')
                else:
                    Logger.raiseException('Invalid frequency or q value.',exception=ValueError)
                
                #choose panel to plot
                if plotPanel.active==0:
                    s1=graph
                elif plotPanel.active==1:
                    s2=graph
                elif plotPanel.active==2:
                    s3=graph
                elif plotPanel.active==3:
                    s4=graph
                else:
                    s1=graph
                
                
        except ValueError:    #plot a thick red cross on panel
            x1 = np.linspace(0, 10, 100)
            y1 = x1
            y2 = 10-x1
            global x
            x=graph.line(x1, y1, legend="ERROR", line_color="red", line_width=6)
            global xx
            xx=graph.line(x1, y2, legend="ERROR", line_color="red", line_width=6)
            graph.title.text=exitMaterial.value
            graph.title.align="center"
    
    def clear_graph():
        if plot_panel.value=="Plot in 1st panel":
            graph=s1
        elif plot_panel.value=="Plot in 2nd panel":
            graph=s2
        elif plot_panel.value=="Plot in 3rd panel":
            graph=s3
        elif plot_panel.value=="Plot in 4th panel":
            graph=s4
        else:
            graph=s1
        
        if quantity.value=="near-field signal":
            graph.renderers.remove(nfs)
            graph.title.remove()
            graph.legend.remove()
        elif quantity.value=="far-field reflectivity (p-polarized)":
            graph.renderers.remove(fReflecP)
        elif quantity.value=="far-field reflectivity (s-polarized)":
            graph.renderers.remove(fReflecS)
        elif quantity.value=="permittivity":
            graph.renderers.remove(rl)
            graph.renderers.remove(img)
        elif quantity.value=="near-field reflectivity":
            graph.renderers.remove(nReflec)
        else:
            graph.renderers.remove(x)
            graph.renderers.remove(xx)

    run.on_click(calculate_parameters)
    clear.on_click(clear_graph)
    
    menuBox = row(quantityMenu,partMenu)
    SNOMbox = column(row(size, oscAmplitude), row(SNOMqNum, zNum), row(normFreq, normMat),
                  css_classes = ['s0_signal','s1_signal','s2_signal','s3_signal','s4_signal','hidden'])
    
    #Missing commas in the css_classes lists of following Boxes are intentional to ensure that widget display properly. Don't modify!
    qRangeBox = row(minQ, maxQ, qNum,
                  css_classes = ['reflection_coefficient(dispersion)','transmission_coefficient(dispersion)','H_field(dispersion)','Coulomb_kernel(dispersion)'
                                 'reflection_coefficient(single_f)','transmission_coefficient(single_f)',
                                 'H_field(single_f)','Coulomb_kernel(single_f)'])
    
    freqRangeBox = row(minFreq, maxFreq,freqNum,
                css_classes = ['permittivity',
                               'reflection_coefficient(dispersion)','transmission_coefficient(dispersion)','H_field(dispersion)','Coulomb_kernel(dispersion)'
                               'reflection_coefficient(single_q)','transmission_coefficient(single_q)',
                               'H_field(single_q)','Coulomb_kernel(single_q)'
                              's0_signal','s1_signal','s2_signal','s3_signal','s4_signal'])
    
    qValueBox = column(singleFreq,
                  css_classes = ['reflection_coefficient(single_f)','transmission_coefficient(single_f)'
                                 'H_field(single_f)','Coulomb_kernel(single_f)','hidden'])

    freqValueBox = column(singleQ,
                  css_classes = ['reflection_coefficient(single_q)','transmission_coefficient(single_q)'
                                 'H_field(single_q)','Coulomb_kernel(single_q)','hidden'])
    
    indexBox = column(index,css_classes = ['H_field(single_f)','H_field(single_q)','H_field(dispersion)',
                                           'Coulomb_kernel(single_f)','Coulomb_kernel(single_q)',
                                           'Coulomb_kernel(dispersion)','hidden'])
    
    
    quantityValueBox = row(minQuantityValue,maxQuantityValue)
    
    polarizationBox = row(polarizationMenu, 
                          css_classes=[''
                                       'reflection_coefficient(dispersion)','transmission_coefficient(dispersion)'
                                       'reflection_coefficient(single_q)','transmission_coefficient(single_q)'
                                       'reflection_coefficient(single_f)','transmission_coefficient(single_f)'])
    
    lastBox = row(plotPanel,clear,run)

    l = layout(menuBox,polarizationBox,freqRangeBox,SNOMbox,qRangeBox,qValueBox,freqValueBox,indexBox,quantityValueBox,lastBox)
    doc.add_root(l)
    
show(modify_doc)

In [19]:
show(graph)